In [35]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode, b64decode
import json 
import pandas as pd
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('../'))
from dotenv import load_dotenv

load_dotenv()

True

## Import Data

In [36]:
file_queries = os.getenv('REPORT_SERVICE')+ 'server/server-info'
print(file_queries)

http://localhost:8000/report-service/api/v1/server/server-info


In [37]:
from urllib.request import urlopen

# Open the URL using urlopen
with urlopen(file_queries) as response:
    file_contents = response.read().decode('utf-8')
    # print(file_contents)

In [38]:
encryp = {
    "key" : os.environ.get('ENCRYPTION_KEY'),
    "iv" : os.environ.get('ENCRYPTION_IV')
}
print(encryp)

{'key': '1278886821604d2d8567885136565909', 'iv': '3346935883364237'}


In [39]:
def decrypt(encrypted_text):
        # try:
        # print(encrypted_text)
        encrypted_bytes = b64decode(encrypted_text)
        cipher = AES.new(bytes(encryp['key'], 'utf-8'), AES.MODE_CBC, bytes(encryp['iv'],'utf-8'))
        decrypted = cipher.decrypt(encrypted_bytes)
        unpadded = unpad(decrypted, AES.block_size)
        return unpadded 

In [40]:
server_info = decrypt(file_contents).decode()
server_info_json = json.loads(server_info)
print(server_info_json)

{'data': [{'server_name': '360-Webserver-01', 'services_software': 'Frontend , Nginx', 'hostname': 'web1.bigdata.local', 'ip_address': '10.10.10.22'}, {'server_name': '360-Webserver-02', 'services_software': 'Frontend , Nginx', 'hostname': 'web2.bigdata.local', 'ip_address': '10.10.10.23'}, {'server_name': 'Big Data Data node 1', 'services_software': 'DataNode, Impala', 'hostname': 'cdhdn1.bigdata.local', 'ip_address': '10.10.10.14'}, {'server_name': 'Big Data Data Node 2', 'services_software': 'DataNode, Impala', 'hostname': 'cdhdn2.bigdata.local', 'ip_address': '10.10.10.15'}, {'server_name': 'Big Data Data Node 3', 'services_software': 'DataNode, Impala', 'hostname': 'cdhdn3.bigdata.local', 'ip_address': '10.10.10.16'}, {'server_name': 'CDH6-Master-Node-1', 'services_software': 'CM, Hue, Atlas, Ranger, Hive', 'hostname': 'cdhmn1.bigdata.local', 'ip_address': '10.10.10.200'}]}


In [41]:
# Extract the 'data' list from the dictionary
data_list = server_info_json['data']

# Convert the list of dictionaries to a DataFrame
df_server = pd.DataFrame(data_list)

# # Display the DataFrame
# print(df_server)
# df_server

In [42]:
# Add a new column 'Run_Number' with consecutive numbers starting from 1
df_server = df_server.sort_values(by='server_name')
df_server['Run_Number'] =  np.arange(1, len(df_server) + 1)

df_server =  df_server[['Run_Number', 'server_name', 'services_software', 'hostname', 'ip_address']]
df_server  = df_server.rename(columns={"Run_Number": "No", "server_name": "Name","services_software":"Service", "hostname":"Hostname","ip_address":"Private IP"})

# Display the DataFrame with the new column
# print(df_server)

df_server

,No,Name,Service,Hostname,Private IP
0,1,360-Webserver-01,"Frontend , Nginx",web1.bigdata.local,10.10.10.22
1,2,360-Webserver-02,"Frontend , Nginx",web2.bigdata.local,10.10.10.23
3,3,Big Data Data Node 2,"DataNode, Impala",cdhdn2.bigdata.local,10.10.10.15
4,4,Big Data Data Node 3,"DataNode, Impala",cdhdn3.bigdata.local,10.10.10.16
2,5,Big Data Data node 1,"DataNode, Impala",cdhdn1.bigdata.local,10.10.10.14
5,6,CDH6-Master-Node-1,"CM, Hue, Atlas, Ranger, Hive",cdhmn1.bigdata.local,10.10.10.200


In [47]:
server_number = len(df_server)

In [43]:
all_server = df_server.to_dict()
all_server

{'No': {0: 1, 1: 2, 3: 3, 4: 4, 2: 5, 5: 6},
 'Name': {0: '360-Webserver-01',
  1: '360-Webserver-02',
  3: 'Big Data Data Node 2',
  4: 'Big Data Data Node 3',
  2: 'Big Data Data node 1',
  5: 'CDH6-Master-Node-1'},
 'Service': {0: 'Frontend , Nginx',
  1: 'Frontend , Nginx',
  3: 'DataNode, Impala',
  4: 'DataNode, Impala',
  2: 'DataNode, Impala',
  5: 'CM, Hue, Atlas, Ranger, Hive'},
 'Hostname': {0: 'web1.bigdata.local',
  1: 'web2.bigdata.local',
  3: 'cdhdn2.bigdata.local',
  4: 'cdhdn3.bigdata.local',
  2: 'cdhdn1.bigdata.local',
  5: 'cdhmn1.bigdata.local'},
 'Private IP': {0: '10.10.10.22',
  1: '10.10.10.23',
  3: '10.10.10.15',
  4: '10.10.10.16',
  2: '10.10.10.14',
  5: '10.10.10.200'}}

In [55]:
topic_text = f"สถานภาพการใช้งานทรัพยากรและแนวโน้มการใช้งานเครื่องคอมพิวเตอร์แม่ข่าย (Server) จำนวน {server_number} เครื่อง"
topic_text 

'สถานภาพการใช้งานทรัพยากรและแนวโน้มการใช้งานเครื่องคอมพิวเตอร์แม่ข่าย (Server) จำนวน 6 เครื่อง'

In [54]:

context = {
        'topic_text ': topic_text,
         'all_server': all_server
        }

def server_all() :
    # print(context)
    return context